# CNN(Fashion MNIST)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision.transforms import ToTensor

train_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [2]:
batch_size = 100
train_loader = torch.utils.data.DataLoader(
    dataset = train_data,
    batch_size = batch_size,
    shuffle = True,
    drop_last = True
)
test_loader = torch.utils.data.DataLoader(
    dataset = test_data,
    batch_size = batch_size,
    shuffle = False
)

In [3]:
import torch.nn as nn
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(  
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=32*6*6, out_features=64, bias = True)
        self.drop = nn.Dropout2d(0.5)
        self.fc2 = nn.Linear(in_features=64, out_features=32, bias = True)
        self.fc3 = nn.Linear(in_features=32, out_features=10, bias = True)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = nn.functional.relu(x)
        x = self.fc3(x)
        
        return x

In [4]:
model = CNN()

# loss and optimizer
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [5]:
class Print(nn.Module):
    def forward(self, x):
        print(x.size())
        return x

In [6]:
# training
def train(model, criterion, optimizer, data):
    model.train() # training
    for X,y in data:
        y_pred = model(X)
        loss = criterion(y_pred,y)
        
        optimizer.zero_grad() # zero_grad(): 미분값 초기화
        loss.backward() # backpropagation
        optimizer.step() # update weights and biases

In [7]:
def evaluation(model, criterion, optimizer, data):
    model.eval() #evaluation
    test_loss = 0
    accuracy = 0

    with torch.no_grad(): # no update
        for X, y in data:
            y_pred = model(X)
            test_loss += criterion(y_pred, y)
            
            prediction = y_pred.max(1)[1] # max indices
            corrects = (prediction == y)
            accuracy += corrects.sum().float() / float( y.size(0) )
    return test_loss, accuracy        

In [8]:
epoch = 20
for epoch in range(epoch):
    train(model, criterion, optimizer, train_loader)
    
    E, accu = evaluation(model, criterion, optimizer, test_loader)
    print(epoch+1, "loss=", E.item(), "accu=", accu.item())

1 loss= 36.90299987792969 accu= 86.74002075195312
2 loss= 31.972232818603516 accu= 88.40000915527344
3 loss= 30.5328426361084 accu= 89.19000244140625
4 loss= 28.015920639038086 accu= 90.0400161743164
5 loss= 28.3795166015625 accu= 89.83999633789062
6 loss= 27.672943115234375 accu= 89.66000366210938
7 loss= 27.51436996459961 accu= 90.1600112915039
8 loss= 26.226613998413086 accu= 90.31001281738281
9 loss= 26.224544525146484 accu= 90.56998443603516
10 loss= 24.85728645324707 accu= 90.87999725341797
11 loss= 25.225576400756836 accu= 90.82999420166016
12 loss= 29.968059539794922 accu= 88.69001007080078
13 loss= 25.253923416137695 accu= 91.24999237060547
14 loss= 24.24906349182129 accu= 91.23001098632812
15 loss= 24.295604705810547 accu= 91.31998443603516
16 loss= 25.83555793762207 accu= 91.01000213623047
17 loss= 25.035242080688477 accu= 91.30000305175781
18 loss= 24.250638961791992 accu= 91.70999908447266
19 loss= 25.44548797607422 accu= 90.90995788574219
20 loss= 24.43114471435547 accu= 

가끔 local minima에 빠지는 모습을 볼 수 있고, overfitting으로 의심되는 점(혹은 미니배치에 의한 oscillation)으로 볼 수 있는 것은 15번째 이후이다. 15번 정도에서 학습을 멈추면 될 것이다.

## 소감문

글을 읽으면서 과학 철학에 도구론적 반사실론자 중 데이비드 머민의 “shut up and calculate!”라는 말이 떠올랐습니다. 양자역학을 이해하려는 사람에게 한 유명한 말인데, 이 글이 말하고자 하는 내용과 같다고 생각했습니다. 과학은 이 세계를 이해하기 위한 도구이고, 진리라고 생각해서는 안된다는 말이었습니다. 단백질 접힘 구조를 설명하려는 머신러닝 과학자들에 대한 글을 읽으면서, 어떠한 한 방법을 고집하는 것이 아닌 유용한 도구가 있으면 그것을 사용하는 것도 나쁘지 않은 방법이라는 생각이 들었고, 특히 머신러닝은 컴퓨터가 어떻게 학습하는 과정을 정확하게 이해할 수는 없지만, 우리 세계를 설명하기 위한 필수적인 도구라는 생각을 했습니다. 그리고 그러한 머신러닝은 베이커 그룹이 거리 뿐만 아니라 각도까지 더하여 학습시키는 노력이 새로운 알고리즘을 더한 알파폴드2를 따라잡을 수 없듯, 자신이 전통성을 유지하며 원래의 방법을 고수하는 것은 계속 그 자리에 머물렀다는 뜻이 될 수도 있구나 라는 것을 느끼게 되었습니다.

공백 미포함 402자